# Project Titan — YOLO Training (Red Team Edition)

**Estratégia:** Treinar v8n primeiro (rápido), só escalar para v8m se Nano não aprender as bordas douradas.

**Mudanças Red Team aplicadas:**
- Domain Randomization (motion blur, JPEG artifacts, glow, occlusion)
- A/B test: v8n → v8m (escala sob demanda)
- Métricas comparativas automáticas

**Pipeline:**
1. Instalar deps + clonar repo
2. Gerar 5000 imagens com Domain Randomization
3. Treinar YOLOv8n (Nano) — 30 min T4
4. Avaliar: se mAP50 ≥ 0.85 nas cartas hero → DONE
5. Se Nano falhar → treinar YOLOv8m (Medium) — 1.5h T4
6. Comparar e exportar o vencedor

## 0. GPU + Dependências

In [ ]:
!nvidia-smi
import torch
print(f"\nCUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
!pip install -q ultralytics opencv-python-headless numpy tqdm Pillow pyyaml

## 1. Clonar Repo

In [ ]:
import os, glob

REPO_URL = "https://github.com/jeremiasmarinho/botia.git"
REPO_DIR = "/content/botia"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL} {REPO_DIR}
else:
    !cd {REPO_DIR} && git pull

PROJECT_DIR = f"{REPO_DIR}/project_titan"
os.chdir(PROJECT_DIR)
print(f"Dir: {os.getcwd()}")

cards = glob.glob("assets/cards/*.png")
print(f"Card assets: {len(cards)}/52")
if len(cards) < 52:
    print("⚠️  Faltam cards! Execute a célula de upload abaixo.")

In [ ]:
# ── UPLOAD MANUAL (só se cards não vieram no clone) ──
UPLOAD_CARDS = False  # ← Mude para True se precisar

if UPLOAD_CARDS:
    from google.colab import files
    print("Upload cards.zip (52 PNGs):")
    uploaded = files.upload()
    for fname in uploaded:
        if fname.endswith('.zip'):
            !unzip -o {fname} -d assets/cards/
            !rm {fname}
    print(f"Cards: {len(glob.glob('assets/cards/*.png'))}/52")

## 2. Gerar Dados Sintéticos com Domain Randomization

In [ ]:
NUM_IMAGES = 5000
IMGSZ = 640
OUTPUT_DIR = "datasets/synthetic_v3"

# Geração com TODAS as features (~5 min)
!python training/generate_pppoker_data.py \
    --num-images {NUM_IMAGES} \
    --output {OUTPUT_DIR} \
    --imgsz {IMGSZ} \
    --gold-border \
    --showdown \
    --buttons \
    --green-table \
    --domain-rand \
    --domain-rand-pct 0.70 \
    --seed 42

In [ ]:
# Verificar + visualizar
import cv2, matplotlib.pyplot as plt, numpy as np

train_dir = f"{OUTPUT_DIR}/images/train"
val_dir = f"{OUTPUT_DIR}/images/val"
print(f"Train: {len(os.listdir(train_dir))} | Val: {len(os.listdir(val_dir))}")

fig, axes = plt.subplots(2, 4, figsize=(20, 10))
samples = sorted(os.listdir(train_dir))[:8]
for ax, fname in zip(axes.flat, samples):
    img = cv2.cvtColor(cv2.imread(f"{train_dir}/{fname}"), cv2.COLOR_BGR2RGB)
    ax.imshow(img); ax.set_title(fname, fontsize=8); ax.axis('off')
plt.suptitle("Amostras PPPoker + Domain Randomization", fontsize=14)
plt.tight_layout(); plt.show()

## 3. Configurar data.yaml

In [ ]:
import yaml

DATA_YAML = "training/data_colab.yaml"

class_names = {
    0: '2c', 1: '2d', 2: '2h', 3: '2s', 4: '3c', 5: '3d', 6: '3h', 7: '3s',
    8: '4c', 9: '4d', 10: '4h', 11: '4s', 12: '5c', 13: '5d', 14: '5h', 15: '5s',
    16: '6c', 17: '6d', 18: '6h', 19: '6s', 20: '7c', 21: '7d', 22: '7h', 23: '7s',
    24: '8c', 25: '8d', 26: '8h', 27: '8s', 28: '9c', 29: '9d', 30: '9h', 31: '9s',
    32: 'Tc', 33: 'Td', 34: 'Th', 35: 'Ts', 36: 'Jc', 37: 'Jd', 38: 'Jh', 39: 'Js',
    40: 'Qc', 41: 'Qd', 42: 'Qh', 43: 'Qs', 44: 'Kc', 45: 'Kd', 46: 'Kh', 47: 'Ks',
    48: 'Ac', 49: 'Ad', 50: 'Ah', 51: 'As',
    52: 'fold', 53: 'check', 54: 'raise',
    55: 'raise_2x', 56: 'raise_2_5x', 57: 'raise_pot',
    58: 'raise_confirm', 59: 'allin', 60: 'pot', 61: 'stack',
}

train_paths = [f"{OUTPUT_DIR}/images/train"]
val_paths = [f"{OUTPUT_DIR}/images/val"]

if os.path.exists("datasets/titan_cards/images/train"):
    tc = len(os.listdir("datasets/titan_cards/images/train"))
    if tc > 0:
        train_paths.append("datasets/titan_cards/images/train")
        val_paths.append("datasets/titan_cards/images/val")
        print(f"✅ titan_cards: {tc} imagens reais")

if os.path.exists("datasets/synthetic/images/train"):
    sc = len(os.listdir("datasets/synthetic/images/train"))
    if sc > 0:
        train_paths.append("datasets/synthetic/images/train")
        val_paths.append("datasets/synthetic/images/val")
        print(f"✅ synthetic v1: {sc} imagens")

data_config = {
    'path': os.path.abspath('.'),
    'train': train_paths if len(train_paths) > 1 else train_paths[0],
    'val': val_paths if len(val_paths) > 1 else val_paths[0],
    'nc': 62,
    'names': class_names,
}

with open(DATA_YAML, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False, sort_keys=False)

print(f"\n📄 {DATA_YAML} — {len(train_paths)} train sources, 62 classes")

## 4. Fase 1: Treinar YOLOv8n (Nano)

**Hipótese Red Team:** Se o Nano já detecta board cards a 98%, ele TEM capacidade.
O problema é que nunca viu bordas douradas. Se o v3 dataset corrigir isso,
o Nano mantém ~33ms de inferência e a conta de 800ms fecha com folga.

| Modelo | Params | Inferência | Budget Multi-table |
|--------|--------|------------|--------------------|
| **v8n** | 3M | ~33ms | ✅ 6 mesas OK |
| v8m | 25M | ~100ms | ⚠️ 3-4 mesas max |
| v8l | 43M | ~200ms | ❌ 1-2 mesas |

In [ ]:
import time
from ultralytics import YOLO

# ═══════════════════════════════════════════
# FASE 1: YOLOv8n (Nano) — baseline rápido
# ═══════════════════════════════════════════
EPOCHS = 150
BATCH = 16
PATIENCE = 25

print("[FASE 1] Treinando YOLOv8n (Nano)...")
print(f"  Epochs: {EPOCHS} | Batch: {BATCH} | Patience: {PATIENCE}")
print()

model_nano = YOLO("yolov8n.pt")
t0 = time.time()

results_nano = model_nano.train(
    data=os.path.abspath(DATA_YAML),
    epochs=EPOCHS,
    batch=BATCH,
    imgsz=640,
    patience=PATIENCE,
    project="runs/detect",
    name="titan_v7_nano",
    exist_ok=True,
    flipud=0.0,
    fliplr=0.0,
    degrees=5.0,
    mosaic=1.0,
    hsv_h=0.015,
    hsv_s=0.4,
    hsv_v=0.3,
    lr0=0.01,
    lrf=0.01,
    verbose=True,
    plots=True,
)

nano_time = time.time() - t0
print(f"\n[FASE 1] Nano concluído em {nano_time/60:.1f} min")

In [ ]:
# Extrair métricas do Nano
nano_metrics = {}
if results_nano and hasattr(results_nano, 'results_dict'):
    rd = results_nano.results_dict
    nano_metrics = {
        'mAP50': rd.get('metrics/mAP50(B)', 0),
        'mAP50_95': rd.get('metrics/mAP50-95(B)', 0),
        'precision': rd.get('metrics/precision(B)', 0),
        'recall': rd.get('metrics/recall(B)', 0),
    }

print("═" * 50)
print("RESULTADOS FASE 1 — YOLOv8n (Nano)")
print("═" * 50)
for k, v in nano_metrics.items():
    print(f"  {k:12s}: {v:.4f}")
print(f"  {'tempo':12s}: {nano_time/60:.1f} min")
print("═" * 50)

# Decisão automática
NANO_THRESHOLD = 0.85
nano_passed = nano_metrics.get('mAP50', 0) >= NANO_THRESHOLD

if nano_passed:
    print(f"\n✅ Nano mAP50={nano_metrics['mAP50']:.4f} ≥ {NANO_THRESHOLD}")
    print("   → Nano é SUFICIENTE. Não precisa do Medium.")
    print("   → Pule para a Seção 6 (Exportar).")
else:
    print(f"\n⚠️  Nano mAP50={nano_metrics.get('mAP50', 0):.4f} < {NANO_THRESHOLD}")
    print("   → Nano insuficiente. Execute a Fase 2 (Medium) abaixo.")

In [ ]:
# Visualizar predições do Nano
from IPython.display import Image, display

for plot in ["results.png", "confusion_matrix.png", "val_batch0_pred.png"]:
    path = f"runs/detect/titan_v7_nano/{plot}"
    if os.path.exists(path):
        print(f"\n── {plot} ──")
        display(Image(filename=path, width=900))

## 5. Fase 2: Treinar YOLOv8m (Medium) — SÓ SE NANO FALHAR

⚠️ **Execute esta seção apenas se o Nano ficou abaixo de 0.85 mAP50.**

In [ ]:
# ═══════════════════════════════════════════
# FASE 2: YOLOv8m (Medium) — só se necessário
# ═══════════════════════════════════════════
TRAIN_MEDIUM = not nano_passed  # Auto: True se Nano falhou
# TRAIN_MEDIUM = True  # ← Descomente para forçar treino do Medium

if TRAIN_MEDIUM:
    print("[FASE 2] Treinando YOLOv8m (Medium)...")
    model_medium = YOLO("yolov8m.pt")
    t0 = time.time()

    results_medium = model_medium.train(
        data=os.path.abspath(DATA_YAML),
        epochs=EPOCHS,
        batch=BATCH,
        imgsz=640,
        patience=PATIENCE,
        project="runs/detect",
        name="titan_v7_medium",
        exist_ok=True,
        flipud=0.0,
        fliplr=0.0,
        degrees=5.0,
        mosaic=1.0,
        hsv_h=0.015,
        hsv_s=0.4,
        hsv_v=0.3,
        lr0=0.01,
        lrf=0.01,
        verbose=True,
        plots=True,
    )

    medium_time = time.time() - t0
    medium_metrics = {}
    if results_medium and hasattr(results_medium, 'results_dict'):
        rd = results_medium.results_dict
        medium_metrics = {
            'mAP50': rd.get('metrics/mAP50(B)', 0),
            'mAP50_95': rd.get('metrics/mAP50-95(B)', 0),
            'precision': rd.get('metrics/precision(B)', 0),
            'recall': rd.get('metrics/recall(B)', 0),
        }

    print(f"\n[FASE 2] Medium concluído em {medium_time/60:.1f} min")
    print("═" * 50)
    print("RESULTADOS FASE 2 — YOLOv8m (Medium)")
    print("═" * 50)
    for k, v in medium_metrics.items():
        print(f"  {k:12s}: {v:.4f}")
else:
    print("[FASE 2] SKIP — Nano foi suficiente.")
    medium_metrics = {}
    medium_time = 0

In [ ]:
# ═══════════════════════════════════════════
# COMPARAÇÃO A/B
# ═══════════════════════════════════════════
import pandas as pd

comparison = {
    'Modelo': ['YOLOv8n (Nano)', 'YOLOv8m (Medium)'],
    'Params': ['3M', '25M'],
    'Inferência': ['~33ms', '~100ms'],
    'Tempo Treino': [f"{nano_time/60:.1f}m", f"{medium_time/60:.1f}m" if medium_time else "N/A"],
    'mAP50': [
        f"{nano_metrics.get('mAP50', 0):.4f}",
        f"{medium_metrics.get('mAP50', 0):.4f}" if medium_metrics else "N/A"
    ],
    'mAP50-95': [
        f"{nano_metrics.get('mAP50_95', 0):.4f}",
        f"{medium_metrics.get('mAP50_95', 0):.4f}" if medium_metrics else "N/A"
    ],
    'Precision': [
        f"{nano_metrics.get('precision', 0):.4f}",
        f"{medium_metrics.get('precision', 0):.4f}" if medium_metrics else "N/A"
    ],
    'Recall': [
        f"{nano_metrics.get('recall', 0):.4f}",
        f"{medium_metrics.get('recall', 0):.4f}" if medium_metrics else "N/A"
    ],
    'Multi-table 6x': ['✅ OK', '⚠️ 3-4 max'],
}

df = pd.DataFrame(comparison)
print("\n" + "═" * 80)
print("COMPARAÇÃO A/B: Nano vs Medium")
print("═" * 80)
print(df.to_string(index=False))
print("═" * 80)

# Decisão automática
if nano_passed:
    WINNER = "nano"
    print(f"\n🏆 VENCEDOR: YOLOv8n (Nano) — precisão suficiente + latência mínima")
elif medium_metrics:
    WINNER = "medium"
    print(f"\n🏆 VENCEDOR: YOLOv8m (Medium) — Nano insuficiente, Medium necessário")
else:
    WINNER = "nano"
    print(f"\n⚠️ Usando Nano por padrão (Medium não treinado)")

WINNER_DIR = f"runs/detect/titan_v7_{WINNER}"
print(f"   Pesos: {WINNER_DIR}/weights/best.pt")

## 6. Benchmark de Inferência

In [ ]:
# Medir latência real de inferência
import time
import numpy as np

best_model = YOLO(f"{WINNER_DIR}/weights/best.pt")
print(f"Modelo: {WINNER_DIR}")
print(f"Classes: {len(best_model.names)}")
print(f"Params: {sum(p.numel() for p in best_model.model.parameters()):,}")

# Warmup
dummy = np.random.randint(0, 255, (640, 640, 3), dtype=np.uint8)
for _ in range(3):
    best_model.predict(dummy, verbose=False)

# Benchmark 50 inferências
latencies = []
for _ in range(50):
    t0 = time.perf_counter()
    best_model.predict(dummy, verbose=False)
    latencies.append((time.perf_counter() - t0) * 1000)

latencies = np.array(latencies)
print(f"\n── Latência de Inferência ({WINNER.upper()}) ──")
print(f"  Média:  {latencies.mean():.1f} ms")
print(f"  P50:    {np.percentile(latencies, 50):.1f} ms")
print(f"  P95:    {np.percentile(latencies, 95):.1f} ms")
print(f"  P99:    {np.percentile(latencies, 99):.1f} ms")

# Budget check para multi-table
BUDGET_MS = 800
overhead_ms = 150  # OCR + GTO + action + emulador
vision_budget = BUDGET_MS - overhead_ms
tables_possible = int(vision_budget / latencies.mean())
print(f"\n── Budget Multi-Table (total={BUDGET_MS}ms, overhead={overhead_ms}ms) ──")
print(f"  Vision budget: {vision_budget}ms")
print(f"  Mesas possíveis: ~{tables_possible} (sequencial)")

In [ ]:
# Testar em imagens de validação
import cv2
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 4, figsize=(24, 12))
val_images = sorted(os.listdir(val_dir))[:8]

for ax, fname in zip(axes.flat, val_images):
    img_path = f"{val_dir}/{fname}"
    results = best_model.predict(img_path, verbose=False, conf=0.25)
    annotated = results[0].plot()
    ax.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    n_det = len(results[0].boxes) if results[0].boxes is not None else 0
    ax.set_title(f"{fname} ({n_det} det)", fontsize=8)
    ax.axis('off')

plt.suptitle(f"Predições {WINNER.upper()} (Validação)", fontsize=14)
plt.tight_layout(); plt.show()

In [ ]:
# Confusion matrix + curvas
for plot in ["results.png", "confusion_matrix.png"]:
    path = f"{WINNER_DIR}/{plot}"
    if os.path.exists(path):
        print(f"\n── {plot} ──")
        display(Image(filename=path, width=900))

## 7. Exportar Modelo Vencedor

In [ ]:
import shutil, json

best_pt = f"{WINNER_DIR}/weights/best.pt"
last_pt = f"{WINNER_DIR}/weights/last.pt"
best_size = os.path.getsize(best_pt) / (1024 * 1024)
print(f"Modelo: {WINNER} | best.pt: {best_size:.1f} MB")

export_dir = f"/content/titan_v7_export"
os.makedirs(export_dir, exist_ok=True)
shutil.copy2(best_pt, f"{export_dir}/best.pt")
shutil.copy2(last_pt, f"{export_dir}/last.pt")
shutil.copy2(DATA_YAML, f"{export_dir}/data_colab.yaml")

for p in ["results.png", "confusion_matrix.png"]:
    src = f"{WINNER_DIR}/{p}"
    if os.path.exists(src):
        shutil.copy2(src, f"{export_dir}/{p}")

ab_report = {
    'winner': WINNER,
    'nano_metrics': nano_metrics,
    'medium_metrics': medium_metrics if medium_metrics else None,
    'nano_time_min': round(nano_time / 60, 1),
    'medium_time_min': round(medium_time / 60, 1) if medium_time else None,
    'inference_ms_mean': round(float(latencies.mean()), 1),
    'inference_ms_p95': round(float(np.percentile(latencies, 95)), 1),
    'domain_randomization': True,
    'num_images': NUM_IMAGES,
}
with open(f"{export_dir}/ab_test_report.json", 'w') as f:
    json.dump(ab_report, f, indent=2)

zip_path = "/content/titan_v7_weights"
shutil.make_archive(zip_path, 'zip', export_dir)
print(f"\n📦 {zip_path}.zip ({os.path.getsize(zip_path + '.zip') / 1024 / 1024:.1f} MB)")

In [ ]:
from google.colab import files
files.download(f"{zip_path}.zip")

In [ ]:
# (Opcional) Salvar no Google Drive
SAVE_TO_DRIVE = False

if SAVE_TO_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_dir = "/content/drive/MyDrive/titan_models"
    os.makedirs(drive_dir, exist_ok=True)
    shutil.copy2(best_pt, f"{drive_dir}/titan_v7_{WINNER}_best.pt")
    shutil.copy2(f"{zip_path}.zip", f"{drive_dir}/titan_v7_weights.zip")
    print(f"✅ Salvo em: {drive_dir}/")

## 8. Pós-treino

```bash
# 1. Copiar best.pt
copy best.pt project_titan/models/titan_v7.pt

# 2. Atualizar config_club.yaml
vision:
  yolo_model: models/titan_v7.pt

# 3. Testar
python tools/diagnose_vision.py --title "LDPlayer"
python scripts/live_demo.py --title "LDPlayer"
```